# ETL Automation

## Import libraries

In [1]:
import gspread
import gspread_dataframe as gd
import pandas as pd
import numpy as np
import os
import datetime


In [2]:
# os.chdir('/Users/atmavidyavirananda/Desktop/Tableau Public Project')
# os.rename("/Users/atmavidyavirananda/Desktop/ETL Automation.ipynb", "/Users/atmavidyavirananda/Desktop/Tableau Public Project/ETL Automation.ipynb")

## Import data from Github

In [2]:
# Path to Github repository, in which the COVID-19 data is updated daily
main_dir = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series'

confirmed_path = 'time_series_covid19_confirmed_global.csv'
deaths_path = 'time_series_covid19_deaths_global.csv'
recovered_path = 'time_series_covid19_recovered_global.csv'

paths = [confirmed_path, deaths_path, recovered_path]

In [3]:
df_names = ['confirmed_global', 'deaths_global', 'recovered_global']
data = {}

# Get data, create dataframe, then store in a dictionary.
for p,name in zip(paths,df_names):
    data[name] = pd.read_csv(os.path.join(main_dir,p))
    print(f'Extracted {name} data.')
    

Extracted confirmed_global data.
Extracted deaths_global data.
Extracted recovered_global data.


In [5]:
# # Preview data
# print(data['confirmed_global'].shape)
# data['confirmed_global'].head()

## ASEAN Countries

In [5]:
# Manually created list to search for ASEAN countries in the Country/Region column
# Source: https://asean.org/asean/asean-member-states/
asean_list = ['brunei', 'cambodia', 'indonesia', 'lao', 'malaysia', 'myanmar', 'philippines', 'singapore', 'thailand', 'vietnam']

In [6]:
# Empty list to store the actual country name from the Country/Region column
asean_countries = []

In [7]:
# This function will be executed for each value in the Country/Region column
def find_asean(val):
    for country in asean_list:
        if country in val.lower(): # if the current column value matches any of the countries in asean_list
            asean_countries.append(val) # append that value to the asean_countries list

In [8]:
# Execute function with a map method
data['confirmed_global']['Country/Region'].map(find_asean)

0      None
1      None
2      None
3      None
4      None
       ... 
274    None
275    None
276    None
277    None
278    None
Name: Country/Region, Length: 279, dtype: object

In [9]:
# # List of asean country names that is consistent with the names in the original data (Country/Region column)
# asean_countries

In [10]:
# Create a dataframe for confirmed cases, deaths, and recovered that has been filtered for ASEAN countries
asean_confirmed = data['confirmed_global'].loc[data['confirmed_global']['Country/Region'].isin(asean_countries),:]
asean_deaths = data['deaths_global'].loc[data['deaths_global']['Country/Region'].isin(asean_countries),:]
asean_recovered = data['recovered_global'].loc[data['recovered_global']['Country/Region'].isin(asean_countries),:]

In [11]:
# Drop the Province/State since they are missing for every ASEAN country
for df in [asean_confirmed, asean_deaths, asean_recovered]:
    df.drop('Province/State', inplace=True, axis=1)

/Users/atmavidyavirananda/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
# Preview data
asean_confirmed

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21,6/28/21,6/29/21,6/30/21,7/1/21
31,Brunei,4.535300,114.727700,0,0,0,0,0,0,0,...,256,256,256,256,259,259,259,259,260,261
37,Cambodia,11.550000,104.916700,0,0,0,0,0,1,1,...,44124,44711,45366,46065,46810,47649,48532,49255,50385,51384
148,Indonesia,-0.789300,113.921300,0,0,0,0,0,0,0,...,2018113,2033421,2053995,2072867,2093962,2115304,2135998,2156465,2178272,2203108
164,Laos,19.856270,102.495496,0,0,0,0,0,0,0,...,2067,2076,2080,2094,2100,2110,2110,2110,2121,2144
176,Malaysia,4.210484,101.975766,0,0,0,3,4,4,4,...,705762,711006,716847,722659,728462,734048,739266,745703,751979,758967
212,Philippines,12.879721,121.774017,0,0,0,0,0,0,0,...,1367894,1372232,1378260,1385053,1391911,1397992,1403588,1408058,1412559,1418337
230,Singapore,1.283300,103.833300,0,1,3,3,4,5,7,...,62448,62470,62493,62513,62530,62544,62553,62907,62579,62589
248,Thailand,15.870032,100.992541,4,4,5,6,8,8,14,...,225365,228539,232647,236291,240452,244447,249853,254515,259301,264834
274,Vietnam,14.058324,108.277199,0,2,2,2,2,2,2,...,13782,13989,14323,14537,15325,15740,16136,16507,17052,17727


## Transform

If we look at the raw data, we notice that the time series extends horizontally along the table. In other words, the dates are arranged along the columns instead of the rows, which would make it harder to create visualizations in Tableau. This part is intended to illustrate how we would reshape the data such that the time series are presented vertically in one column for every country.

In [13]:
# declare parameters for melting the dataframe
dim_cols = data['confirmed_global'].columns[1:4]
ts_cols = data['confirmed_global'].columns[4:]

In [14]:
metrics = ['Confirmed', 'Deaths', 'Recovered'] 
rsh = {} # dictionary of reshaped dataframes

# Loop through each dataframe and transform, then store in the designated dictionary.
for df,m in zip([asean_confirmed, asean_deaths, asean_recovered], metrics):
    rsh[m] = df.melt(id_vars=dim_cols, value_vars=ts_cols, var_name='Date', value_name = m)

### Combine the 3 data sources into 1 dataframe

In [15]:
all_data = rsh['Confirmed'].sort_values(by=['Country/Region','Confirmed']).copy()

In [16]:
all_data['Deaths'] = rsh['Deaths'].sort_values(by=['Country/Region','Deaths'])['Deaths']
all_data['Recovered'] = rsh['Recovered'].sort_values(by=['Country/Region','Recovered'])['Recovered']

In [17]:
all_data['Date'] = pd.to_datetime(all_data['Date'])

In [18]:
all_data.rename(columns={'Country/Region':'Country'}, inplace=True)

In [19]:
# country_loc = all_data.groupby('Country/Region').max()[['Lat','Long']].reset_index()

## Add new columns for actual daily cases

Since the original data is presented as a cummulative, in order to get the actual daily metrics (confirmed, deaths, and recovered cases ONLY in that specific date) we would need to subtract the current date value with the previous date value. We do this by lagging the 3 metrics forward by 1 day, then subtracting the current date value with that lagged value (of course, this is done by grouping the countries in advance). 

In SQL, this would be equivalent to applying a window function on the 3 metrics. Assuming the table is named all_data, the query would be something similar to this:

----
    SELECT 
        `Country`,
        `Lat`,
        `Long`,
        `Date`,
        `Confirmed`,
        `Deaths`,
        `Recovered`,
        `Confirmed` - COALESCE(`lagged_confirmed`,0) as `Actual Confirmed`,
        `Deaths` - COALESCE(`lagged_deaths`,0) as `Actual Deaths`,
        `Recovered` - COALESCE(`lagged_recovered`,0) as `Actual Recovered`
    FROM (
        SELECT *,
               lag(`Confirmed`, 1) over(partition by `Country` order by `Date`) as `lagged_confirmed`,
               lag(`Deaths`, 1) over(partition by `Country` order by `Date`) as `lagged_deaths`,
               lag(`Recovered`, 1) over(partition by `Country` order by `Date`) as `lagged_recovered`
        FROM all_data
        )t
-----

Additional note: the COALESCE function is used to fill the NaN values - caused by lagging the first value forward - with zero.


In [20]:
# Adding the lagged columns
all_data['lagged_confirmed'] = all_data.groupby(by='Country')['Confirmed'].shift(1)
all_data['lagged_deaths'] = all_data.groupby(by='Country')['Deaths'].shift(1)
all_data['lagged_recovered'] = all_data.groupby(by='Country')['Recovered'].shift(1)

In [21]:
# Filling the NaN values with zero
for col in ['lagged_confirmed', 'lagged_deaths', 'lagged_recovered']:
    all_data[col] = all_data[col].fillna(0)

In [22]:
# Subtracting to get the actual daily cases
all_data['Actual Confirmed'] = all_data['Confirmed']- all_data['lagged_confirmed']
all_data['Actual Deaths'] = all_data['Deaths']- all_data['lagged_deaths']
all_data['Actual Recovered'] = all_data['Recovered']- all_data['lagged_recovered']

In [29]:
# Check the result
all_data.loc[all_data['Country']=='Indonesia',:].tail(10)

,Country,Lat,Long,Date,Confirmed,Deaths,Recovered,lagged_confirmed,lagged_deaths,lagged_recovered,Actual Confirmed,Actual Deaths,Actual Recovered
4619,Indonesia,-0.7893,113.9213,2021-06-18,1963266,54043,1779127,1950276.0,53753.0,1771220.0,12990.0,290.0,7907.0
4628,Indonesia,-0.7893,113.9213,2021-06-19,1976172,54291,1786143,1963266.0,54043.0,1779127.0,12906.0,248.0,7016.0
4637,Indonesia,-0.7893,113.9213,2021-06-20,1989909,54662,1792528,1976172.0,54291.0,1786143.0,13737.0,371.0,6385.0
4646,Indonesia,-0.7893,113.9213,2021-06-21,2004445,54956,1801761,1989909.0,54662.0,1792528.0,14536.0,294.0,9233.0
4655,Indonesia,-0.7893,113.9213,2021-06-22,2018113,55291,1810136,2004445.0,54956.0,1801761.0,13668.0,335.0,8375.0
4664,Indonesia,-0.7893,113.9213,2021-06-23,2033421,55594,1817303,2018113.0,55291.0,1810136.0,15308.0,303.0,7167.0
4673,Indonesia,-0.7893,113.9213,2021-06-24,2053995,55949,1826504,2033421.0,55594.0,1817303.0,20574.0,355.0,9201.0
4682,Indonesia,-0.7893,113.9213,2021-06-25,2072867,56371,1835061,2053995.0,55949.0,1826504.0,18872.0,422.0,8557.0
4691,Indonesia,-0.7893,113.9213,2021-06-26,2093962,56729,1842457,2072867.0,56371.0,1835061.0,21095.0,358.0,7396.0
4700,Indonesia,-0.7893,113.9213,2021-06-27,2115304,57138,1850481,2093962.0,56729.0,1842457.0,21342.0,409.0,8024.0


## Export to Google Sheets

In [30]:
# Use Google Sheet API credentials to authorize access 
gc = gspread.service_account('/Users/atmavidyavirananda/Desktop/tableau-public-viz-3d037cec5382.json')

In [31]:
# Access the premade sheet (make sure the generated credential is given editor access to the sheet)
wks = gc.open('ASEAN COVID-19 Data - Daily')
ts_data = wks.worksheet('Time series')
log = wks.worksheet('Log')

In [37]:
# Write time series data
ts_data.clear()
gd.set_with_dataframe(ts_data, all_data)

In [38]:
# # Write latest data
# max_date = all_data['Date'].max()
# latest = all_data.loc[all_data['Date']==max_date,:]

# latest_data.clear()
# gd.set_with_dataframe(latest_data, latest)

In [39]:
# Update log
curr_timestamp = datetime.datetime.now().strftime('%D %H:%M:%S')
log.update('A1', curr_timestamp)

{'spreadsheetId': '1iDQuZfghpW784NYimKboTVEKRS3YwMoHUxygNhw3eTg',
 'updatedRange': 'Log!A1',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}